In [11]:
# --- Imports ---
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
import joblib
import json


# --- 1. Load saved model and scaler ---
model = load_model("success_predictor_model.keras")


scaler = joblib.load("scaler.pkl")

# Load y_min and y_max for denormalizing the prediction
with open("y_norm_range.json") as f:
    y_min, y_max = json.load(f)

# --- 2. Define new event inputs ---

# 🔢 Numeric features (replace these with your real values)
# --- Define 3 events: bad, average, great ---

new_event_numeric = pd.DataFrame([
   # BAD (~0.25–0.30)
    {
        "rating": 2.9,
        "total_events": 25,
        "avg_checkins": 16.5,
        "avg_likes": 33.0,
        "avg_event_rating": 3.0,
        "median_event_rating": 3.1,
        "venue_popularity_tier": 1,
        "checkin_count": 12,
        "like_count": 25,
        "rating_avg": 2.2,
        "rating_count": 10,
        "engagement": 26.4,  # = 26.4
        "event_weekday": 1,
        "event_month": 2,
        "start_hour": 14,
        "price_rating_encoded": 1,
        
    },
    # AVERAGE (~0.50)
    {
       "rating": 3.6,
        "total_events": 36,
        "avg_checkins": 19.5,
        "avg_likes": 38.5,
        "avg_event_rating": 3.4,
        "median_event_rating": 3.5,
        "venue_popularity_tier": 3,
        "checkin_count": 26,
        "like_count": 48,
        "rating_avg": 3.5,
        "rating_count": 24,
        "engagement": 91.0,  # = 91.0
        "event_weekday": 3,
        "event_month": 6,
        "start_hour": 20,
        "price_rating_encoded": 1,
    },

      # GREAT (~0.90+)
    {
        "rating": 5.0,
        "total_events": 55,
        "avg_checkins": 22.0,
        "avg_likes": 44.0,
        "avg_event_rating": 3.7,
        "median_event_rating": 3.9,
        "venue_popularity_tier": 5,
        "checkin_count": 43,
        "like_count": 72,
        "rating_avg": 4.6,
        "rating_count": 33,
        "engagement": 197.8,  # = 197.8
        "event_weekday": 5,
        "event_month": 8,
        "start_hour": 21,
        "price_rating_encoded": 1,
       
    }
])[[
    'rating', 'total_events', 'avg_checkins', 'avg_likes', 'avg_event_rating',
    'median_event_rating', 'venue_popularity_tier', 'checkin_count', 'like_count',
    'rating_avg', 'rating_count','engagement', 'event_weekday', 'event_month', 'start_hour',
    'price_rating_encoded'
]]


# 🏷️ Tag IDs (must be padded to length 6)
tag_lists = [
    [50],      # Bad: None, Any
    [10, 11],     # Average: Acoustic, Indie
    [3, 4, 5]     # Great: Rap, Rave, Live
]

# -- 1. Pad tag input (already correct) --
max_len = 6
tag_input = np.array([
    tags + [0] * (max_len - len(tags)) for tags in tag_lists
])

# -- 2. Scale numeric input once --
scaler = joblib.load("scaler.pkl")
new_numeric_scaled = scaler.transform(new_event_numeric)

# -- 3. Predict with correct input shapes --
pred_scaled = model.predict({
    "numeric_input": new_numeric_scaled,
    "tag_input": tag_input
}).flatten()

# -- 4. Denormalize --
pred_scores = pred_scaled * (y_max - y_min) + y_min

# -- 5. Print results --
for i, score in enumerate(pred_scores):
    label = ["💀 BAD", "😐 AVERAGE", "🌟 GREAT"][i]
    print(f"{label} Event → Predicted Success Score: {score:.4f}")


ValueError: Input 0 of layer "functional_25" is incompatible with the layer: expected shape=(None, 15), found shape=(3, 16)